In [1]:
import sys

sys.path.append("..")

In [2]:
from create_maxim_model import Model
from maxim.configs import MAXIM_CONFIGS

2022-10-01 12:19:05.108512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# From https://github.com/google-research/maxim/blob/main/maxim/run_eval.py#L55

VARIANT = "S-3"
configs = MAXIM_CONFIGS.get(VARIANT)

configs.update(
    {
        "variant": VARIANT,
        "dropout_rate": 0.0,
        "num_outputs": 3,
        "use_bias": True,
        "num_supervision_scales": 3,
    }
)

model = Model(**configs)

2022-10-01 12:19:08.347836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# From https://github.com/google-research/maxim/blob/main/maxim/run_eval.py

import tensorflow as tf
import numpy as np
import collections 
import io


def recover_tree(keys, values):
    """Recovers a tree as a nested dict from flat names and values.
    This function is useful to analyze checkpoints that are saved by our programs
    without need to access the exact source code of the experiment. In particular,
    it can be used to extract an reuse various subtrees of the scheckpoint, e.g.
    subtree of parameters.
    Args:
      keys: a list of keys, where '/' is used as separator between nodes.
      values: a list of leaf values.
    Returns:
      A nested tree-like dict.
    """
    tree = {}
    sub_trees = collections.defaultdict(list)
    for k, v in zip(keys, values):
        if "/" not in k:
            tree[k] = v
        else:
            k_left, k_right = k.split("/", 1)
            sub_trees[k_left].append((k_right, v))
    for k, kv_pairs in sub_trees.items():
        k_subtree, v_subtree = zip(*kv_pairs)
        tree[k] = recover_tree(k_subtree, v_subtree)
    return tree


def get_params(ckpt_path):
    """Get params checkpoint."""

    with tf.io.gfile.GFile(ckpt_path, "rb") as f:
        data = f.read()
    values = np.load(io.BytesIO(data))
    params = recover_tree(*zip(*values.items()))
    params = params["opt"]["target"]

    return params

In [5]:
CKPT_PATH = "gs://gresearch/maxim/ckpt/Denoising/SIDD/checkpoint.npz"
jax_params = get_params(CKPT_PATH)

In [6]:
model.count_params() / 1e6

22.212795

In [7]:
list(jax_params.keys())[:5]

['UpSampleRatio_0',
 'UpSampleRatio_1',
 'UpSampleRatio_10',
 'UpSampleRatio_11',
 'UpSampleRatio_12']

In [8]:
def get_model_vars(model):
    model_variables = model.variables
    model_variables_dict = {}
    for v in model_variables:
        model_variables_dict[v.name] = v

    return model_variables_dict

In [9]:
tf_params = get_model_vars(model)

In [10]:
list(tf_params.keys())[:5]

['stage_0_input_conv_0/kernel:0',
 'stage_0_input_conv_0/bias:0',
 'stage_0_encoder_block_0_conv_in/kernel:0',
 'stage_0_encoder_block_0_conv_in/bias:0',
 'stage_0_encoder_block_0_SplitHeadMultiAxisGmlpLayer_0_LayerNorm_in/gamma:0']

In [11]:
jax_params.keys()

dict_keys(['UpSampleRatio_0', 'UpSampleRatio_1', 'UpSampleRatio_10', 'UpSampleRatio_11', 'UpSampleRatio_12', 'UpSampleRatio_13', 'UpSampleRatio_14', 'UpSampleRatio_15', 'UpSampleRatio_16', 'UpSampleRatio_17', 'UpSampleRatio_18', 'UpSampleRatio_19', 'UpSampleRatio_2', 'UpSampleRatio_20', 'UpSampleRatio_21', 'UpSampleRatio_22', 'UpSampleRatio_23', 'UpSampleRatio_24', 'UpSampleRatio_25', 'UpSampleRatio_26', 'UpSampleRatio_27', 'UpSampleRatio_28', 'UpSampleRatio_29', 'UpSampleRatio_3', 'UpSampleRatio_30', 'UpSampleRatio_31', 'UpSampleRatio_32', 'UpSampleRatio_33', 'UpSampleRatio_34', 'UpSampleRatio_35', 'UpSampleRatio_36', 'UpSampleRatio_37', 'UpSampleRatio_38', 'UpSampleRatio_39', 'UpSampleRatio_4', 'UpSampleRatio_40', 'UpSampleRatio_41', 'UpSampleRatio_42', 'UpSampleRatio_43', 'UpSampleRatio_44', 'UpSampleRatio_45', 'UpSampleRatio_46', 'UpSampleRatio_47', 'UpSampleRatio_48', 'UpSampleRatio_49', 'UpSampleRatio_5', 'UpSampleRatio_50', 'UpSampleRatio_51', 'UpSampleRatio_52', 'UpSampleRatio_